<a href="https://colab.research.google.com/github/coder-apr-5/GenAI_EXCELR/blob/main/Text_to_Speech.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers sentencepiece gtts ipywidgets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.2/98.2 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 23.9 MB/s eta 0:00:00
  Attempting uninstall: click
    Found existing installation: click 8.2.1
    Uninstalling click-8.2.1:
      Successfully uninstalled click-8.2.1


In [2]:
!pip install SpeechRecognition pydub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 32.9/32.9 MB 50.8 MB/s eta 0:00:00


In [3]:
from transformers import MarianMTModel,MarianTokenizer
from gtts import gTTS
import speech_recognition as sr
from pydub import AudioSegment
from IPython.display import Audio,display,clear_output
from google.colab import files
import ipywidgets as widgets
import os
import warnings
warnings.filterwarnings("ignore")

In [4]:
language_code_map = {

'English': 'en',
'French': 'fr',
'German': 'de',
'Spanish': 'es',
'Hindi': 'hi',
'Russian': 'ru',
'Chinese': 'zh',
'Japanese': 'ja',
'Arabic': 'ar',
'Portuguese': 'pt',
 'Telugu': 'te',
 'Urdu': 'ur',
 'Malayalam': 'ml',
 'Tamil': 'ta',
 'Kannada': 'kn',
 'Gujarati': 'gu',
 'Marathi': 'mr',
 'Punjabi': 'pa',
 'Assamese': 'as',
 'Nepali': 'ne',
 'Oriya': 'or',
 'Bengali': 'bn',
}

In [5]:
model_name_map = {

('en', 'fr'): 'Helsinki-NLP/opus-mt-en-fr',
('fr', 'en'): 'Helsinki-NLP/opus-mt-fr-en',
('en', 'de'): 'Helsinki-NLP/opus-mt-en-de',
('de', 'en'): 'Helsinki-NLP/opus-mt-de-en',
('en', 'es'): 'Helsinki-NLP/opus-mt-en-es',
('es', 'en'): 'Helsinki-NLP/opus-mt-es-en',
('en', 'pt'):'Helsinki-NLP/opus-mt-en-pt',
('en', 'hi'): 'Helsinki-NLP/opus-mt-en-hi',
('en', 'ru'): 'Helsinki-NLP/opus-mt-en-ru',
('en', 'zh'): 'Helsinki-NLP/opus-mt-en-zh',
('en', 'ja'): 'Helsinki-NLP/opus-mt-en-ja',
('en', 'ar'): 'Helsinki-NLP/opus-mt-en-ar',
('en', 'ur'): 'Helsinki-NLP/opus-mt-en-ur',
 ('en', 'zh'): 'Helsinki-NLP/opus-mt-en-zh',
 ('en', 'zh-tw'): 'Helsinki-NLP/opus-mt-en-zh',
 ('en', 'ko'): 'Helsinki-NLP/opus-mt-en-ko',
 ('en', 'te'): 'Helsinki-NLP/opus-mt-en-te',
 ('en', 'ur'): 'Helsinki-NLP/opus-mt-en-ur',
 ('en', 'ml'): 'Helsinki-NLP/opus-mt-en-ml',
 ('en', 'ta'): 'Helsinki-NLP/opus-mt-en-ta',
 ('en', 'kn'): 'Helsinki-NLP/opus-mt-en-kn',
 ('en', 'gu'): 'Helsinki-NLP/opus-mt-en-gu',
 ('en', 'mr'): 'Helsinki-NLP/opus-mt-en-mr',
 ('en', 'pa'): 'Helsinki-NLP/opus-mt-en-pa',
 ('en', 'as'): 'Helsinki-NLP/opus-mt-en-as',
 ('en', 'ne'): 'Helsinki-NLP/opus-mt-en-ne',
 ('en', 'or'): 'Helsinki-NLP/opus-mt-en-or',
 ('en', 'bn'): 'Helsinki-NLP/opus-mt-en-bn',
}

In [6]:
def translate_text(text, src_lang, tgt_lang):
  """Translate text using MarianMT models"""
  lang_pair = (src_lang, tgt_lang)
  if lang_pair not in model_name_map:
    raise ValueError(f"No model available for translation from {src_lang} to {tgt_lang}")

  model_name = model_name_map[lang_pair]
  tokenizer = MarianTokenizer.from_pretrained(model_name)
  model = MarianMTModel.from_pretrained(model_name)

  inputs = tokenizer(text, return_tensors="pt",padding=True , truncation=True,max_length=512)
  translated = model.generate(**inputs)
  translated_text = tokenizer.decode(translated[0], skip_special_tokens=True)
  return translated_text

In [7]:
def speak_text(text, lang_code, speed=1):
  """Convert text to speech with speed control"""
  try:
    tts = gTTS(text=text, lang=lang_code, slow=True if speed == 2 else False)
    audio_path= "output.mp3"
    tts.save(audio_path)
    display(Audio(audio_path, autoplay=True))
    display(widgets.Button(description="Download Audio",
                           button_style='success',
                           on_click=lambda: files.download(audio_path)))
  except Exception as e:
    print(f"TTT Error:{str(e)}")

In [8]:
def recognize_speech(audio_file):
  """Convert speech to text using Google Speech Recognition"""
  recognizer = sr.Recognizer()
  if audio_file.endswith('.mp3'):
    audio = AudioSegment.from_mp3(audio_file)
    wav_file = audio_file.replace('.mp3', '.wav')
    audio.export(wav_file, format='wav')
  else:
    wav_file = audio_file
  try:
    with sr.AudioFile(wav_file) as source:
      audio_data = recognizer.record(source)
      return recognizer.recognize_google(audio_data)
  except Exception as e:
    print(f"Recognition Error:{str(e)}")
    return None

In [9]:
source_dropdown = widgets.Dropdown(
    options=list(language_code_map.keys()),
    value='English',
    description='Source Language:',
    style = {'description_width': 'initial'}
)
target_dropdown = widgets.Dropdown(
    options=list(language_code_map.keys()),
    value='French',
    description='Target Language:',
    style = {'description_width': 'initial'}
)

speed_slider = widgets.IntSlider(
    value=1,
    min=1,
    max=2,
    step=1,
    description='Speed:',
    style = {'description_width': 'initial'}
)

input_selector = widgets.RadioButtons(
    options=['Text', 'Speech'],
    value='Text',
    description='Input Type:',

)

text_input = widgets.Textarea(

    placeholder='Enter text to translate',
    layout=widgets.Layout(width='80%', height='100px')
)

file_upload = widgets.FileUpload(
    accept='.mp3,.wav',
    multiple=False,
    description='Upload Audio',

)
execute_button = widgets.Button(
    description='Translate & Speak',
    button_style='success',
    icon='play'
)

output_area=widgets.Output()

In [10]:
def run_translator():
  display(widgets.HBox([source_dropdown, target_dropdown, speed_slider]))
  display(input_selector)

  def on_execute(b):
    clear_output(wait=True)
    src_lang = language_code_map[source_dropdown.value]
    tgt_lang = language_code_map[target_dropdown.value] # Corrected target language

    text = None # Initialize text variable
    if input_selector.value == 'Text':
        display(text_input)
        text = text_input.value
    elif input_selector.value == 'Speech':
        display(file_upload)
        if file_upload.value:
            uploaded_file = list(file_upload.value.keys())[0]
            with open(uploaded_file, 'wb') as f:
                f.write(file_upload.value[uploaded_file]['content'])
            text = recognize_speech(uploaded_file)
            if text:
                print(f"Recognized Text: {text}")
            else:
                print("Could not recognize speech.")
                return
        else:
            print("Please upload an audio file.")
            return

    if text: # Only proceed if text is available
      try:
        translated = translate_text(text, src_lang, tgt_lang)
        print(f"Translated Text: {translated}")
        speak_text(translated, tgt_lang, speed_slider.value)
      except Exception as e:
        print(f"Translation Error: {str(e)}")
    else:
        print("No text to translate.")

  execute_button= widgets.Button(description="Translate & Speak")
  execute_button.on_click(on_execute)

  if input_selector.value == 'Text':
    display(text_input)
  else:
    display(file_upload)

  display(execute_button)

run_translator()

RadioButtons(description='Input Type:', options=('Text', 'Speech'), value='Text')

Textarea(value='', layout=Layout(height='100px', width='80%'), placeholder='Enter text to translate')

Button(description='Translate & Speak', style=ButtonStyle())

In [11]:
def run_translator():
  display(widgets.HBox([source_dropdown, target_dropdown, speed_slider]))
  display(input_selector)

  def on_execute(b):
    clear_output(wait=True)
    src_lang = language_code_map[source_dropdown.value]
    tgt_lang = language_code_map[target_dropdown.value] # Corrected target language

    text = None # Initialize text variable
    if input_selector.value == 'Text':
        display(text_input)
        text = text_input.value
    elif input_selector.value == 'Speech':
        display(file_upload)
        if file_upload.value:
            uploaded_file = list(file_upload.value.keys())[0]
            with open(uploaded_file, 'wb') as f:
                f.write(file_upload.value[uploaded_file]['content'])
            text = recognize_speech(uploaded_file)
            if text:
                print(f"Recognized Text: {text}")
            else:
                print("Could not recognize speech.")
                return
        else:
            print("Please upload an audio file.")
            return

    if text: # Only proceed if text is available
      try:
        translated = translate_text(text, src_lang, tgt_lang)
        print(f"Translated Text: {translated}")
        speak_text(translated, tgt_lang, speed_slider.value)
      except Exception as e:
        print(f"Translation Error: {str(e)}")
    else:
        print("No text to translate.")

  execute_button= widgets.Button(description="Translate & Speak")
  execute_button.on_click(on_execute)

  if input_selector.value == 'Text':
    display(text_input)
  else:
    display(file_upload)

  display(execute_button)

run_translator()

RadioButtons(description='Input Type:', options=('Text', 'Speech'), value='Text')

Textarea(value='', layout=Layout(height='100px', width='80%'), placeholder='Enter text to translate')

Button(description='Translate & Speak', style=ButtonStyle())

In [12]:
def batch_translate(texts, src_lang, tgt_lang):
  translator = get_translator(src_lang, tgt_lang)
  return [out['translation_text']for out in translator(texts)]

In [13]:
def safe_translate(text, src, tgt):
  try:
    if len(text) > 5000:
      return "Error : Text too long(max 5000 characters)"
    if src == tgt:
      return text
    if (src , tgt) not in model_name_map:
      return"Error : Unsupported language pair"
    return translate_text(text, src, tgt)
  except Exception as e:
    return f"Translation Error : {str(e)}"

In [14]:
def on_upload_change(change):
  """Handle file upload changes."""
  if file_upload.value:
    with output_area:
      clear_output()
      print("Processing audio file...")
      # Access the content directly from the change dictionary
      uploaded_file_content = list(change['new'].values())[0]['content']

      # Create a temporary file to pass to recognize_speech
      temp_audio_path = "temp_audio.wav"
      with open(temp_audio_path, 'wb') as f:
          f.write(uploaded_file_content)

      text = recognize_speech(temp_audio_path)

      # Remove the temporary file
      os.remove(temp_audio_path)

      if text:
        text_input.value = text # Assign the recognized text to the text_input widget
        print(f"Recognized Text: {text}")
      else:
        print("Could not recognize speech.")

def on_execute(b):
  """Main translation function"""
  with output_area:
    clear_output()
    src_lang = language_code_map[source_dropdown.value]
    tgt_lang = language_code_map[target_dropdown.value]
    text = None # Initialize text variable

    if input_selector.value == 'Text':
        text = text_input.value.strip()
        if not text:
            print("Please enter text to translate.")
            return
    elif input_selector.value == 'Speech':
        if file_upload.value:
            # The on_upload_change function should have already populated text_input.value
            text = text_input.value.strip()
            if not text:
                print("No speech recognized. Please try again.")
                return
        else:
            print("Please upload an audio file.")
            return

    try:
      display(widgets.HTML("<i>Translating...</i>"))
      translated = translate_text(text, src_lang, tgt_lang)
      display(widgets.HTML(
          f"<h3>Translation ({target_dropdown.value}):</h3>"
          f"<p style='font-size:16px'>{translated}</p>"
      ))
      speak_text(translated, tgt_lang, speed_slider.value)
    except Exception as  e:
      print(f"Error: {str(e)}")

# Observe file_upload changes only once when the UI is created
# file_upload.observe(on_upload_change, names='value')
# execute_button.on_click(on_execute) # This is already handled in create_ui

In [15]:
def run_translator(b):
  with output_area:
    clear_output()
    try:
      src_lang_name = source_dropdown.value
      tgt_lang_name = target_dropdown.value
      src_lang_code = language_code_map[src_lang_name]
      tgt_lang_code = language_code_map[tgt_lang_name]
      speed = speed_slider.value
      input_type = input_selector.value

      if input_type == 'Text':
        text_to_translate = text_input.value
        if not text_to_translate:
          print("Please enter text to translate.")
          return
        translated_text = translate_text(text_to_translate, src_lang_code, tgt_lang_code)
        print("Translated Text:")
        print(translated_text)
        speak_text(translated_text, tgt_lang_code, speed)

      elif input_type == 'Speech':
        if not file_upload.value:
          print("Please upload an audio file.")
          return
        uploaded_file = list(file_upload.value.values())[0]
        file_content = uploaded_file['content']
        file_name = uploaded_file['name']
        with open(file_name, 'wb') as f:
          f.write(file_content)

        recognized_text = recognize_speech(file_name)
        if recognized_text:
          print("Recognized Text:")
          print(recognized_text)
          translated_text = translate_text(recognized_text, src_lang_code, tgt_lang_code)
          print("Translated Text:")
          print(translated_text)
          speak_text(translated_text, tgt_lang_code, speed)
        else:
          print("Could not recognize speech.")

    except ValueError as e:
        print(f"Error: {e}")
    except Exception as e:
        print(f"An unexpected error occurred: {e}")

execute_button.on_click(run_translator)

In [16]:
def create_ui():
  """Create responsive UI layout"""
  input_box = widgets.VBox([text_input])

  def update_input(change):
    if change['new'] == 'Text':
      input_box.children = [text_input]
    elif change['new'] == 'Speech': # Corrected indentation
      input_box.children = [file_upload]

  input_selector.observe(update_input, names='value')

  app = widgets.VBox([
      widgets.HTML("<h1 style='text-align:center'>Multilingual Translator</h1>"),
      widgets.HBox([
          source_dropdown,
          target_dropdown,
          speed_slider,
          input_selector
      ]),
      input_box,
      execute_button,
      output_area
  ], layout=widgets.Layout(
      width='90%',
      margin='0 auto',
      border='1px solid #ccc',
      padding='20px'
  ))

  return app

display(create_ui())